# Скачивание и предобработка текстов

Скачаем тексты сочинений с сайта https://mogu-pisat.ru/sochinenie/.

In [1]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
from datetime import datetime

In [2]:
session = requests.session()

In [3]:
!pip install pyyaml ua-parser user-agents fake-useragent


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
ua = UserAgent(verify_ssl=False)

In [5]:
pages = []

for n in range(2, 50):
    link = 'https://mogu-pisat.ru/sochinenie/ege/?PAGEN_1={}'.format(n)
    headers = {'User-Agent': ua.random}
    response = session.get(link, headers=headers).text
    pages.append(response)
    print(datetime.now())
    time.sleep(1)

2023-10-22 03:37:54.627838
2023-10-22 03:37:55.977414
2023-10-22 03:37:57.291984
2023-10-22 03:37:58.644125
2023-10-22 03:37:59.998704
2023-10-22 03:38:01.385610
2023-10-22 03:38:02.718850
2023-10-22 03:38:04.036128
2023-10-22 03:38:05.385605
2023-10-22 03:38:06.766609
2023-10-22 03:38:08.127017
2023-10-22 03:38:09.455138
2023-10-22 03:38:10.826659
2023-10-22 03:38:12.212420
2023-10-22 03:38:13.531954
2023-10-22 03:38:14.847107
2023-10-22 03:38:16.142561
2023-10-22 03:38:17.454428
2023-10-22 03:38:18.761417
2023-10-22 03:38:20.142642
2023-10-22 03:38:21.460346
2023-10-22 03:38:22.780571
2023-10-22 03:38:24.143203
2023-10-22 03:38:25.504399
2023-10-22 03:38:26.814627
2023-10-22 03:38:28.121665
2023-10-22 03:38:29.481785
2023-10-22 03:38:30.808164
2023-10-22 03:38:32.120281
2023-10-22 03:38:33.497066
2023-10-22 03:38:34.825705
2023-10-22 03:38:36.201322
2023-10-22 03:38:37.506971
2023-10-22 03:38:38.842339
2023-10-22 03:38:40.164016
2023-10-22 03:38:41.475903
2023-10-22 03:38:42.782983
2

In [6]:
len(pages)

48

In [7]:
links = []

for page in pages:
    soup = BeautifulSoup(page, 'html.parser')
    new_links = soup.find_all('div', {'class' : "b-news-item-name"})
    for i in new_links:
        one = i.find('a').get('href')
        links.append(one)

In [8]:
len(links)

960

С помощью краулинга мы собрали ссылки на 960 сочинений. Учитывая, что размер сочинений, как правило, составляет около 200 слов, скачаем 90 сочинений.

In [10]:
essays = dict()
ids = 0

for l in links[:90]:
    full_link = 'https://mogu-pisat.ru' + l
    headers = {'User-Agent': ua.random}
    response = session.get(full_link, headers=headers).text
    soup = BeautifulSoup(response, 'html.parser')
    text = soup.find('div', {"style" : "font-size:17px;"}).get_text()
    title = soup.title.get_text()
    author = soup.find_all('div', {"style" : "font-size:16px;"})[1].get_text()
    data = [text, title, author, full_link]
    essays.update({ids : data})
    ids += 1
    print(datetime.now())
    time.sleep(1)

2023-10-22 03:41:09.546648
2023-10-22 03:41:10.793802
2023-10-22 03:41:12.111769
2023-10-22 03:41:13.370746
2023-10-22 03:41:14.611988
2023-10-22 03:41:15.882446
2023-10-22 03:41:17.110674
2023-10-22 03:41:18.332151
2023-10-22 03:41:19.651773
2023-10-22 03:41:20.900133
2023-10-22 03:41:22.210598
2023-10-22 03:41:23.467399
2023-10-22 03:41:24.706851
2023-10-22 03:41:25.941195
2023-10-22 03:41:27.235779
2023-10-22 03:41:28.502851
2023-10-22 03:41:29.737932
2023-10-22 03:41:30.957737
2023-10-22 03:41:32.579936
2023-10-22 03:41:34.072002
2023-10-22 03:41:35.777488
2023-10-22 03:41:37.545361
2023-10-22 03:41:38.848486
2023-10-22 03:41:40.088643
2023-10-22 03:41:41.390184
2023-10-22 03:41:42.608648
2023-10-22 03:41:43.908547
2023-10-22 03:41:45.161449
2023-10-22 03:41:46.418646
2023-10-22 03:41:47.635349
2023-10-22 03:41:48.932401
2023-10-22 03:41:50.173985
2023-10-22 03:41:51.466459
2023-10-22 03:41:52.720076
2023-10-22 03:41:54.023215
2023-10-22 03:41:55.257275
2023-10-22 03:41:56.510603
2

In [11]:
len(essays)

90

Essays - словарь, в котором ключами являются id сочинений, а значениями - информация о них: 
1. текст сочинения
2. название сочинения
3. автор
4. ссылка на сайт

Сохраним получившийся корпус в файл формата json:

In [13]:
import json
with open('corpus.json', 'w', encoding="utf-8") as fp:
    json.dump(essays, fp, ensure_ascii=False)

Разделим сочинения на предложения:

In [14]:
from nltk import sent_tokenize

def cleaning(text):
    cleaning = "\n\xa0«»\t—…" + "'‘’"
    new_text = ''

    for ch in text:
        if ch not in cleaning:
            new_text += ch
    sent_tokens = sent_tokenize(new_text)
    return sent_tokens

In [15]:
id_sent = 0
sentences = dict()

for essay in essays.items():
    key = essay[0]
    content = essay[1][0]
    sents = cleaning(content)
    for s in sents:
        sent_doc = [s, key]
        sentences.update({id_sent : sent_doc})
        id_sent += 1

In [16]:
len(sentences)

1527

Мы создали словарь, в котором ключами являются id предложений. Значения - текст предложения и id сочинения, из которого оно было извлечено. 

Длина корпуса - 1527 предложений

In [17]:
sentences[0]

[' В предложенном для анализа тексте Владимир Николаевич Крупин размышляет о взаимодействии природы и человека.',
 0]

In [18]:
with open('corpus_sent.json', 'w', encoding="utf-8") as f:
    json.dump(sentences, f, ensure_ascii=False)

# Частеречная разметка корпуса

Для морфологической разметки корпуса было решено использовать библиотеку pymorphy по нескольким причинам:
* относительно высокая точноть разметки и производительность
* удобный тегсет, специализированный для русского языка
* несколько вариантов морфологического разбора с вероятностью каждого -> возможность разрешать или не разрешать морфологическую неоднозначность в особых случаях

In [19]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [20]:
from nltk import word_tokenize

**Разрешение морфологической неоднозначности**

Было принято решение в отдельных случаях в морфологической разметке приписывать одной и той же словоформе нескольких разборов. К этим случаям относятся ситуации, когда определение части речи зависит от контекста, ср.:
* *так* - наречие или частица
* формы прилагательных, совпадающие с формой причастий (*открытый* и под.)

Pymorphy указывает вероятность того, что тот или иной разбор правильный (score). В случаях, когда score >= 0,6, словоформе приписывает один, наиболее вероятный тег. В остальных случаях словоформе приписывается несколько возможных разборов.

In [21]:
def tagging(word):
    parsed = morph.parse(word.lower())
    wordform = parsed[0].word
    lemma = [parsed[0].normal_form]
    pos = [str(parsed[0].tag).split(',')[0].split(' ')[0]]
    if parsed[0].score < 0.6:
        for option in parsed:
            other_tag = str(option.tag).split(',')[0].split(' ')[0]
            if other_tag not in pos:
                pos.append(other_tag)
                lemma.append(option.normal_form)
        return wordform, lemma, pos
    else:
        return wordform, lemma, pos

In [22]:
tokens = dict()
token_id = 0

for sent in sentences.items():
    sent_id = sent[0]
    sentence = sent[1][0]
    words = word_tokenize(sentence)
    for w in words:
        word_info = tagging(w)
        num_tags = len(word_info[2])
        if num_tags > 1:
            for i in range(num_tags):
                info = {'wordform' : word_info[0],
                        'lemma' : word_info[1][i],
                        'pos' : word_info[2][i]}
                tokens.update({token_id : info})
                token_id += 1
        else:
            info = {'wordform' : word_info[0],
                    'lemma' : word_info[1][0],
                    'pos' : word_info[2][0]}
            tokens.update({token_id : info})
            token_id += 1

**Корпус токенов**

Мы создали словарь, в котором ключами являются token_id. По каждому ключу выдаётся следующая информация о словоформе:
* id предложения, в котором она содержится
* сама словоформа
* лемма
* pos

В случаях, когда словоформа имеет несколько вариантов разбора, каждый разбор хранится под отедльным token_id.

Словоформа с единственным разбором:

In [23]:
tokens[0]

{'wordform': 'в', 'lemma': 'в', 'pos': 'PREP'}

Словоформа с двумя вариантами разбора:

In [24]:
print(tokens[23])
print(tokens[24])

{'wordform': 'следующие', 'lemma': 'следующий', 'pos': 'ADJF'}
{'wordform': 'следующие', 'lemma': 'следовать', 'pos': 'PRTF'}


In [25]:
len(tokens)

32009

Всего корпус содержит 32009 токенов (включая пунктуацию).

In [26]:
with open('corpus_tokens.json', 'w', encoding='utf-8') as file:
    json.dump(tokens, file, ensure_ascii=False)